## Deliverable 2. Create a Customer Travel Destinations Map.

In [33]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Dikson,RU,73.5069,80.5464,47.95,87,51,13.56,broken clouds
1,1,Winneba,GH,5.3511,-0.6231,72.16,85,52,5.84,broken clouds
2,2,Castro,BR,-24.7911,-50.0119,59.20,85,100,1.95,overcast clouds
3,3,San Quintin,MX,30.4833,-115.9500,75.13,61,66,17.56,broken clouds
4,4,Ancud,CL,-41.8697,-73.8203,40.23,94,100,7.09,light rain


In [18]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 77
What is the maximum temperature you would like for your trip? 91


In [19]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Vaini,TO,-21.2000,-175.2000,78.96,83,20,9.22,few clouds
10,10,Kapaa,US,22.0752,-159.3190,80.19,76,10,5.01,clear sky
11,11,Saint Anthony,US,45.0205,-93.2180,80.53,48,40,6.91,scattered clouds
15,15,Georgetown,MY,5.4112,100.3354,85.93,86,20,5.75,few clouds
24,24,Butaritari,KI,3.0707,172.7902,81.14,77,29,6.87,light rain
26,26,Katsuura,JP,35.1333,140.3000,84.22,86,54,17.09,broken clouds
27,27,Atuona,PF,-9.8000,-139.0333,77.36,71,3,15.05,clear sky
35,35,Kavieng,PG,-2.5744,150.7967,82.40,76,100,11.10,light rain
44,44,Tiarei,PF,-17.5333,-149.3333,78.82,76,7,12.57,light rain
47,47,Nago,JP,26.5881,127.9761,90.03,76,14,12.37,few clouds


In [20]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                196
City                   196
Country                196
Lat                    196
Lng                    196
Max Temp               196
Humidity               196
Cloudiness             196
Wind Speed             196
Current Description    196
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

# There are no empty rows.


In [22]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Vaini,TO,78.96,few clouds,-21.2000,-175.2000,
10,Kapaa,US,80.19,clear sky,22.0752,-159.3190,
11,Saint Anthony,US,80.53,scattered clouds,45.0205,-93.2180,
15,Georgetown,MY,85.93,few clouds,5.4112,100.3354,
24,Butaritari,KI,81.14,light rain,3.0707,172.7902,
26,Katsuura,JP,84.22,broken clouds,35.1333,140.3000,
27,Atuona,PF,77.36,clear sky,-9.8000,-139.0333,
35,Kavieng,PG,82.40,light rain,-2.5744,150.7967,
44,Tiarei,PF,78.82,light rain,-17.5333,-149.3333,
47,Nago,JP,90.03,few clouds,26.5881,127.9761,


In [23]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
     hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [28]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   196
Country                196
Max Temp               196
Current Description    196
Lat                    196
Lng                    196
Hotel Name             196
dtype: int64

In [29]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [34]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)

# 11a. Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>